In [ ]:
import gmsh
import os
import import_ipynb
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import mu_0
from fem_pre_processing import read_mesh
from fem_processing import gaussian_quadrature, matrices_assembly
from fem_pos_processing import fem_analysis as fem, graph_results as graph

# Project 2: Magnetic Shielding, Spherical Shell of Permeable Material in a Uniform Field

# `apply_physics()`

In [ ]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    for key, cell in cell_data.items():  
        # Adicionar as propriedades do materiais ao dicionário da célula
        cell['stiffness_term'] = []
        cell['mass_term'] = []
        cell['source'] = []
        
        # Calcular o valor de 'a' de acordo com a física do problema
        mur = cell['material']['relative_magnetic_permeability']
        #sigma = cell['material']['electric_conductivity']
        a = 1/(mur * mu_0)
        
        # Get the global coordinates of the nodes
        ai = [nodes_data[idx]['xg'] for idx in cell['conn']]

        # Get the Gauss points and weights
        gauss_points, _ = gaussian_quadrature.gauss_data(FINITE_ELEMENT)

        # Adicionar a nova chave 'material' ao dicionário da célula
        for xik in gauss_points: 
            xg_e, yg_e = matrices_assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, ai, xik)
            cell['stiffness_term'].append(a * np.eye(2))
            cell['mass_term'].append(1)           
        
            # Adicionar a nova chave 'source' ao dicionário da célula
            cell['source'].append(0)

        # Dictionary with boundary nodes
        cell_data[key]['abc'] = {'type': None, 'conn_idx': None}

    return mesh_data

# `create_entire_domain()`

In [ ]:
def create_entire_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=True, view_mesh=False):
    mesh_data = {}
    type, order = FINITE_ELEMENT

    # Dimensões do domínio
    width = 0.5
    height = 0.5
    ra = 0.1
    rb = ra + 0.02205

    # Tamanhos de malha
    max_size = h        # Tamanho no restante do domínio
    min_size = h * 0.3  # Tamanho mínimo nas linhas refinadas e na casca

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("axial_magnetic_shell")
    factory = gmsh.model.geo
        
    # Criação dos pontos
    p1 = factory.addPoint(0, 0, 0)  # Inferior esquerdo
    p2 = factory.addPoint(width, 0, 0)  # Inferior direito
    p3 = factory.addPoint(width, height, 0)  # Superior direito
    p4 = factory.addPoint(0, height, 0)  # Superior esquerdo

    # Pontos da casca esférica
    p11 = factory.addPoint(ra, 0, 0)  
    p12 = factory.addPoint(rb, 0, 0)  
    p41 = factory.addPoint(0, rb, 0)  
    p42 = factory.addPoint(0, ra, 0)  

    # Criação dos domínios Neumann Homogêneo Inferior
    l11 = factory.addLine(p1, p11)
    l12 = factory.addLine(p11, p12)
    l13 = factory.addLine(p12, p2)

    # Criação do domínio Dirichlet não-homogêneo
    l2 = factory.addLine(p2, p3)  

    # Superior
    l3 = factory.addLine(p3, p4)

    # Criação do domínio Dirichlet homogêneo
    l41 = factory.addLine(p4, p41)
    l42 = factory.addLine(p41, p42)          
    l43 = factory.addLine(p42, p1)

    # Linhas da casca esférica
    inner_shell = factory.addCircleArc(p11, p1, p42)
    outer_shell = factory.addCircleArc(p12, p1, p41)

    # Criar o contorno da superfície
    air_loop = factory.addCurveLoop([l11, inner_shell, l43])
    magnetic_loop = factory.addCurveLoop([l12, outer_shell, l42, -inner_shell])
    free_loop = factory.addCurveLoop([l13, l2, l3, l41, -outer_shell])

    air_cavity = factory.addPlaneSurface([air_loop])
    magnetic_shell = factory.addPlaneSurface([magnetic_loop])
    free_space = factory.addPlaneSurface([free_loop])

    # Refinamento: aplicar campos de distância para a casca esférica
    gmsh.model.mesh.field.add("Distance", 1)
    gmsh.model.mesh.field.setNumbers(1, "EdgesList", [inner_shell, outer_shell])  # Refinamento nos arcos da casca esférica

    # Refinamento adicional para a linha l41
    gmsh.model.mesh.field.add("Distance", 2)
    gmsh.model.mesh.field.setNumbers(2, "EdgesList", [l41])

    # # Refinamento adicional para a linha l2
    # gmsh.model.mesh.field.add("Distance", 3)
    # gmsh.model.mesh.field.setNumbers(3, "EdgesList", [l2])

    # Refinamento adicional para a linha l11
    gmsh.model.mesh.field.add("Distance", 4)
    gmsh.model.mesh.field.setNumbers(4, "EdgesList", [l11])

    # Refinamento adicional para a linha l43
    gmsh.model.mesh.field.add("Distance", 5)
    gmsh.model.mesh.field.setNumbers(5, "EdgesList", [l43])

    # Combinação de refinamentos
    gmsh.model.mesh.field.add("Min", 6)
    #gmsh.model.mesh.field.setNumbers(6, "FieldsList", [1, 2, 3, 4, 5])
    gmsh.model.mesh.field.setNumbers(6, "FieldsList", [1, 2, 4, 5])

    # Campo Threshold para controlar o refinamento
    gmsh.model.mesh.field.add("Threshold", 7)
    gmsh.model.mesh.field.setNumber(7, "InField", 6)
    gmsh.model.mesh.field.setNumber(7, "SizeMin", min_size)  # Tamanho mínimo nas linhas e casca
    gmsh.model.mesh.field.setNumber(7, "SizeMax", max_size)  # Tamanho máximo fora das regiões refinadas
    gmsh.model.mesh.field.setNumber(7, "DistMin", 0.01)  # Distância mínima para aplicar o tamanho min_size
    gmsh.model.mesh.field.setNumber(7, "DistMax", 0.05)  # Distância máxima para aplicar o tamanho max_size

    # Aplicar o campo de malha combinado
    gmsh.model.mesh.field.setAsBackgroundMesh(7)

    # Gerar a malha
    factory.synchronize()

    # Definindo as curvas de contorno de Dirichlet (dim=1)
    gmsh.model.addPhysicalGroup(dim=1, tags=[l41, l42, l43], tag=BOUNDARY[0]['tag'], name=BOUNDARY[0]['name'])
    gmsh.model.addPhysicalGroup(dim=1, tags=[l2], tag=BOUNDARY[1]['tag'], name=BOUNDARY[1]['name'])

    # Adicionar grupos físicos para Dim=2 (superfícies)
    gmsh.model.addPhysicalGroup(dim=2, tags=[air_cavity], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])
    gmsh.model.addPhysicalGroup(dim=2, tags=[magnetic_shell], tag=MATERIAL[1]['tag'], name=MATERIAL[1]['name'])
    gmsh.model.addPhysicalGroup(dim=2, tags=[free_space], tag=MATERIAL[2]['tag'], name=MATERIAL[2]['name'])

    # Sincronizar e gerar malha
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/axial_magnetic_shell_domain_{type}{order}.msh"
        print(f"Malha salva em {file_path}")
        gmsh.write(file_path)
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Apply physics to the problem
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)
    
    gmsh.finalize()

    return mesh_data

# `set_analytical_solution()`

In [ ]:
def set_analytical_solution(x, y):
    return 0, np.zeros((2,1))

# `plot_b_field()`

In [ ]:
def plot_b_field(FINITE_ELEMENT, mesh_data, Azh, domain='Entire', flag_quiver=False):
    # Extração de informações sobre o elemento finito
    type, order = FINITE_ELEMENT
    
    # Estrutura de dados
    xg, yg, Bx, By, B_abs = [], [], [], [], []

    # Get the Gauss points and weights
    gauss_points, _ = gaussian_quadrature.general_triangle_rule(m=1, p=1)

    for cell in mesh_data['cell'].values():
        # Get the global coordinates of the nodes
        aie = [mesh_data['nodes'][idx]['xg'] for idx in cell['conn']]
        
        for xik in gauss_points:
            # Coordenadas globais no ponto de Gauss
            xge, yge = matrices_assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, aie, xik)

            # Gradiente da solução aproximada u_h no ponto de Gauss
            grad_uh = fem.grad_uh_at_xik(FINITE_ELEMENT, mesh_data, cell, xik, Azh)

            # Rotacional do potencial vetor magnético
            Bfield = np.array([[0, -1], [1, 0]]) @ grad_uh

            # Acumular listas para visualização
            xg.append(xge)
            yg.append(yge)
            Bx.append(Bfield[0])
            By.append(Bfield[1])
            B_abs.append(np.linalg.norm(Bfield))

    # Visualização do campo magnético
    plt.figure(figsize=(10, 8))

    # Plotar o campo magnético
    plt.tricontourf(xg, yg, B_abs, levels=50, cmap='Greys', alpha=0.6, zorder=0)
    plt.colorbar(label=r'$|\mathbf{B}| (T)$')
    filepath = graph.get_dir(f"pos_processing/pictures/b_field_{type}{order}_{domain}Domain.svg")
    
    if flag_quiver:
        plt.quiver(xg, yg, Bx, By, angles='xy', scale_units='xy',
                    scale=80, color='blue', zorder=1)
        filepath = graph.get_dir(f"pos_processing/pictures/b_field_quiver_{type}{order}_{domain}Domain.svg")
    
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()
    plt.axis('equal')
    
    # Salvando o arquivo no formato SVG
    print(f"Arquivo salvo em: {filepath}")
    plt.savefig(filepath, format="svg")
    plt.close()

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.